# 🏠 Ev Fiyat Tahmin Projesi - Feature Engineering ve Veri Ön İşleme

Bu notebook, eksik değer doldurma ve basit özellik türetme adımlarını içerir.


In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
print("✅ Kütüphaneler yüklendi")


✅ Kütüphaneler yüklendi


In [2]:
from ml_utils import (
    outlier_thresholds, replace_with_thresholds,
    quick_missing_imp, rare_encoder,
    grab_col_names, label_encoder, one_hot_encoder,
    target_mean_encode, plot_importance
)
print("ml_utils import edildi")


ml_utils import edildi


In [3]:
train = pd.read_csv('datasets/housePrice/house_price_train.csv')
test = pd.read_csv('datasets/housePrice/house_price_test.csv')
df = pd.concat([train, test], ignore_index=True)
print(df.shape)
df.head(2)


(2919, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0


In [4]:
# Eksik değer doldurma: sayısal = median (SalePrice HARİÇ), kategorik = mode
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c != 'SalePrice']
cat_cols = df.select_dtypes(include=['object']).columns

if len(num_cols):
    df[num_cols] = SimpleImputer(strategy='median').fit_transform(df[num_cols])
if len(cat_cols):
    df[cat_cols] = SimpleImputer(strategy='most_frequent').fit_transform(df[cat_cols])

print("Eksik değer doldurma (SalePrice hariç) tamamlandı")
df.isnull().sum().sum()


Eksik değer doldurma (SalePrice hariç) tamamlandı


1459

In [5]:
# Basit özellikler
if {'1stFlrSF','2ndFlrSF','TotalBsmtSF'}.issubset(df.columns):
    df['TotalSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['TotalBsmtSF']

print("Yeni sütunlar:", [c for c in df.columns if c.endswith('SF')][-5:])
df[['TotalSF']].head() if 'TotalSF' in df.columns else df.head(1)


Yeni sütunlar: ['2ndFlrSF', 'LowQualFinSF', 'WoodDeckSF', 'OpenPorchSF', 'TotalSF']


,TotalSF
0,2566.0
1,2524.0
2,2706.0
3,2473.0
4,3343.0


In [6]:
# Notebook çıktısı olarak sadece df şekli gösterelim (kaydetme opsiyonel)
print("Güncel şekil:", df.shape)


Güncel şekil: (2919, 82)


## 🧱 Aykırı Değer (Outlier) İşleme
Sayısal değişkenlerde IQR yöntemine göre alt/üst sınırları kullanarak kırpma (winsorize) uygularız.


In [7]:
for col in num_cols:
    if col != 'SalePrice':
        replace_with_thresholds(df, col)

print('Aykırı değer baskılama tamamlandı (0.10-0.90 IQR yöntemi)')


Aykırı değer baskılama tamamlandı (0.10-0.90 IQR yöntemi)


In [8]:
no_cols = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu',
           'GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
for col in [c for c in no_cols if c in df.columns]:
    df[col] = df[col].fillna('No')
print('"No" ile doldurulan sütunlar işlendi')


"No" ile doldurulan sütunlar işlendi


In [9]:
df = quick_missing_imp(df, num_method='median', cat_length=17)
print('Eksik değer doldurma (quick_missing_imp) tamamlandı')


Eksik değer doldurma (quick_missing_imp) tamamlandı


In [10]:
df = rare_encoder(df, rare_perc=0.01)
print('Rare encoder uygulandı')


Rare encoder uygulandı


## 🧩 Hedefe Duyarlı Kategorik Kodlama (Target Mean Encoding)
Sınırlı örnekleme hatasından kaçınmak için basit smoothing ile ortalama kodlama uygularız.


In [11]:
# Ek özellikler (script ile hizalı)
if {'1stFlrSF','GrLivArea'}.issubset(df.columns):
    df['NEW_1st*GrLiv'] = df['1stFlrSF'] * df['GrLivArea']
if {'GarageArea','GrLivArea'}.issubset(df.columns):
    df['NEW_Garage*GrLiv'] = df['GarageArea'] * df['GrLivArea']

qual_cols = ["OverallQual", "OverallCond", "ExterQual", "ExterCond", "BsmtCond", "BsmtFinType1",
             "BsmtFinType2", "HeatingQC", "KitchenQual", "Functional", "FireplaceQu", "GarageQual", "GarageCond", "Fence"]
numeric_qual_cols = [col for col in qual_cols if col in df.columns and df[col].dtype in ['int64','float64']]
df['TotalQual'] = df[numeric_qual_cols].sum(axis=1) if numeric_qual_cols else 0

if {'GrLivArea','TotalBsmtSF'}.issubset(df.columns):
    df['NEW_TotalSqFeet'] = df['GrLivArea'] + df['TotalBsmtSF']
if {'GarageArea','LotArea'}.issubset(df.columns):
    df['NEW_GarageLotRatio'] = df['GarageArea'] / df['LotArea']
if {'MasVnrArea','NEW_TotalHouseArea'}.issubset(df.columns):
    df['NEW_MasVnrRatio'] = df['MasVnrArea'] / df['NEW_TotalHouseArea']
if {'LotArea','1stFlrSF','GarageArea','NEW_PorchArea','WoodDeckSF'}.issubset(df.columns):
    df['NEW_DifArea'] = (df['LotArea'] - df['1stFlrSF'] - df['GarageArea'] - df['NEW_PorchArea'] - df['WoodDeckSF'])
if {'OverallQual','OverallCond'}.issubset(df.columns):
    df['NEW_OverallGrade'] = df['OverallQual'] * df['OverallCond']
if {'YrSold','GarageYrBlt'}.issubset(df.columns):
    df['NEW_GarageSold'] = df['YrSold'] - df['GarageYrBlt']

print('Ek özellikler eklendi')


Ek özellikler eklendi


In [12]:
# Drop list (script ile hizalı)
drop_list = ['Street','Alley','LandContour','Utilities','LandSlope','Heating','PoolQC','MiscFeature','Neighborhood']
df = df.drop([c for c in drop_list if c in df.columns], axis=1)
print('Gereksiz sütunlar düşürüldü')


Gereksiz sütunlar düşürüldü


In [13]:
# Encoding (Label + One-Hot) — ml_utils sürümü
cat_cols, cat_but_car, num_cols = grab_col_names(df)

binary_cols = [col for col in df.columns if df[col].dtype == 'O' and df[col].nunique() == 2]
for col in binary_cols:
    df = label_encoder(df, col)

df = one_hot_encoder(df, cat_cols, drop_first=True)
print('Encoding tamamlandı. Son şekil:', df.shape)


Encoding tamamlandı. Son şekil: (2919, 186)


In [14]:
# Yalnızca train kısmında SalePrice mevcut olduğu için yalnız train'de fit edip df'e merge edeceğiz
train_len = len(train)
train_part = df.iloc[:train_len].copy()

for col in ['MSZoning','Neighborhood','HouseStyle']:
    if col in df.columns and 'SalePrice' in train_part.columns:
        mapping = target_mean_encode(train_part, col)
        df[f'TME_{col}'] = df[col].map(mapping)

print("Target mean encoding (smoothing) uygulandı")


Target mean encoding (smoothing) uygulandı


## 🔧 Gelişmiş Feature Engineering
`house_prediction.py`'daki mantığa paralel örnekler: toplam kat alanı, toplam bodrum alanı, oranlar ve yaş değişkenleri.


In [15]:
# Benzer türetmeler (var olan sütunlara göre korumalı koşullar)
if {'1stFlrSF','2ndFlrSF'}.issubset(df.columns):
    df['NEW_TotalFlrSF'] = df['1stFlrSF'] + df['2ndFlrSF']
if {'BsmtFinSF1','BsmtFinSF2'}.issubset(df.columns):
    df['NEW_TotalBsmtFin'] = df['BsmtFinSF1'] + df['BsmtFinSF2']
if {'OpenPorchSF','EnclosedPorch','ScreenPorch','3SsnPorch','WoodDeckSF'}.issubset(df.columns):
    df['NEW_PorchArea'] = df['OpenPorchSF'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['3SsnPorch'] + df['WoodDeckSF']
if {'YearRemodAdd','YearBuilt'}.issubset(df.columns):
    df['NEW_Restoration'] = df['YearRemodAdd'] - df['YearBuilt']
if {'YrSold','YearBuilt'}.issubset(df.columns):
    df['NEW_HouseAge'] = df['YrSold'] - df['YearBuilt']
if {'YrSold','YearRemodAdd'}.issubset(df.columns):
    df['NEW_RestorationAge'] = df['YrSold'] - df['YearRemodAdd']
if {'GarageYrBlt','YearBuilt'}.issubset(df.columns):
    df['NEW_GarageAge'] = df['GarageYrBlt'] - df['YearBuilt']
if {'GarageYrBlt','YearRemodAdd'}.issubset(df.columns):
    df['NEW_GarageRestorationAge'] = (df['GarageYrBlt'] - df['YearRemodAdd']).abs()

# Oranlar
if {'GrLivArea','LotArea'}.issubset(df.columns):
    df['NEW_LotRatio'] = df['GrLivArea'] / df['LotArea']
if {'NEW_TotalFlrSF','TotalBsmtSF','LotArea'}.issubset(df.columns):
    df['NEW_TotalHouseArea'] = df['NEW_TotalFlrSF'] + df['TotalBsmtSF']
    df['NEW_RatioArea'] = df['NEW_TotalHouseArea'] / df['LotArea']


## 💾 İşlenmiş Veriyi Kaydet (Opsiyonel)
Modelleme notebook'u için CSV olarak kaydedebilirsiniz.


In [16]:
df.to_csv('processed_data.csv', index=False)
print('processed_data.csv kaydedildi. Şekil:', df.shape)


processed_data.csv kaydedildi. Şekil: (2919, 194)
